In [1]:
'''
Идея: сначала отсеять все сомнительные действия – для этого важна полнота 1
Это нужно, чтобы сбалансировать данные
После этого можно обучить модель
'''

'\nИдея: сначала отсеять все сомнительные действия –\xa0для этого важна полнота 1\nЭто нужно, чтобы сбалансировать данные\nПосле этого можно обучить модель\n'

In [20]:
from datetime import datetime
from collections import Counter, defaultdict

import pandas as pd

def convert_date(timestamp):
    return datetime.strptime(timestamp, '%m/%d/%Y %H:%M:%S')

In [2]:
train_df = pd.read_csv('../datasets/r3.1.csv')

/tmp/ipykernel_4677/3051281496.py:1: DtypeWarning: Columns (3,4,9,10,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../datasets/r3.1.csv')


In [3]:
train_df.head()

,id,date,user,Connect,Disconnect,is_new_pc,avg_daily_device_connection,device_connection_num,device_con_above_avg,pc,...,Logoff,Logon,filename,content,avg_daily_file_usage,file_usage_num,file_above_avg,before_avg_logon,after_avg_logout,target
0,{I5K5-M8HD47HI-6694GOIH},01/02/2010 07:13:39,NLR0174,True,False,False,2.573467,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.707113,0.000000,False
1,{F7F1-T1QC34XE-2487PQDL},01/02/2010 07:42:43,JZA0447,True,False,False,8.620520,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.778338,0.000000,False
2,{H6A7-G6LI73WO-1485RABK},01/02/2010 07:48:19,CTR0215,True,False,False,8.652888,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,56.883929,False
3,{M8G7-R4HQ13GC-1992PQQB},01/02/2010 07:50:00,CTR0215,False,True,False,8.652888,2.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,176.883929,False
4,{I2H7-J2VT44PH-7817GZEE},01/02/2010 07:51:43,JZA0447,False,True,False,8.620520,2.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,370.429648,False


In [4]:
# Бдуем рассматривать только атаки по первому сценарию (в тестовом датасете это пользователь CSF0929)

train_df['target'] = train_df.apply(
    lambda x: x['target'] and x['user'] == 'CSF0929',
    axis=1,
)

In [5]:
Counter(train_df['target'])

Counter({False: 1717185, True: 32})

In [6]:
# Соотношение 32 / 1717004 очень плохое, предлагается откинуть все действия, которые не вызывают подозрений:
# 1. Действие выполняется между типисным временем входы и типичным временем выхода

In [13]:
filtered_test_df = train_df[
    train_df.apply(
        lambda x: (
            x['before_avg_logon'] > 60
            or x['after_avg_logout'] > 60
            or x['device_con_above_avg'] > 0
            or x['file_above_avg'] > 0
            or False
        ),
        axis=1,
    )
]

In [14]:
Counter(filtered_test_df['target'])

Counter({False: 1443277, True: 30})

In [10]:
train_df[train_df.apply(lambda x: x['target'], axis=1)].head(32)

,id,date,user,Connect,Disconnect,is_new_pc,avg_daily_device_connection,device_connection_num,device_con_above_avg,pc,...,Logoff,Logon,filename,content,avg_daily_file_usage,file_usage_num,file_above_avg,before_avg_logon,after_avg_logout,target
158491,{V1J9-P7JR21ZY-0017JSES},07/01/2010 02:23:04,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1178.678082,0.00000,True
158492,{Q7Q0-F7FL92IO-3645ARQH},07/01/2010 03:53:50,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,419.62585,True
158493,{O4Y3-X5SF91ZE-7867MGPF},07/01/2010 04:09:15,CSF0929,True,False,False,1.8,3.0,1.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.678082,0.00000,True
158494,{V6B0-T8YP88MD-6528ZNPI},07/01/2010 05:50:23,CSF0929,False,True,False,1.8,4.0,2.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,241.62585,True
160798,{F9X7-E1CP85QD-3175HSBP},07/02/2010 21:57:07,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,677.62585,True
160800,{V2Q2-P1JW27KX-4514LWVQ},07/02/2010 22:40:25,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.678082,0.00000,True
163567,{O5Y9-E5LL15ZL-7586ILYR},07/08/2010 06:18:28,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1474.678082,0.00000,True
163568,{M0A0-U5AK71ET-3193XUDJ},07/08/2010 06:46:19,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,2.62585,True
164726,{N6J8-T9UV76LX-9928WBKB},07/09/2010 01:07:57,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2139.678082,0.00000,True
164728,{T0R8-Z6OF30MJ-4136OHNN},07/09/2010 02:51:26,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,298.62585,True


In [16]:
set(train_df[train_df.apply(lambda x: x['target'], axis=1)]['id']) - set(filtered_test_df[filtered_test_df.apply(lambda x: x['target'], axis=1)]['id'])

{'{E9P9-G5RW21EF-9610NUTR}', '{N5T7-C5QJ36OV-6256GORU}'}

In [22]:
full_train_dataset = pd.read_csv('../data/r3.1/device.csv')

In [23]:
full_train_dataset = full_train_dataset[full_train_dataset.apply(lambda x: x['user'] == 'CCH0959', axis=1)]

In [24]:
monthly_device_usage = defaultdict(int)

for _, row in full_train_dataset.iterrows():
    dt = convert_date(row['date'])
    monthly_device_usage[(dt.year, dt.month)] += 1

monthly_device_usage

defaultdict(int,
            {(2010, 1): 85,
             (2010, 2): 88,
             (2010, 3): 107,
             (2010, 4): 97,
             (2010, 5): 82,
             (2010, 6): 91,
             (2010, 7): 69,
             (2010, 8): 90,
             (2010, 9): 181})